In [35]:
from seleniumbase import SB
import pandas as pd

In [36]:
'''
Sempre que for rodar o código da célula abaixo de 
novo, rode essa célula para resetar as variáveis.
'''
anos = [2023, 2024]
links = {}
df_fbref_competition = {}
df_fbref_teams = {}
teaminfo = {}

In [37]:
# 2 = two years, 1 = one year

competitions_format = {
    "Premier League": "2",
    "La Liga": "2",
    "Ligue 1": "2",
    "Fußball-Bundesliga": "2",
    "Serie A": "2",
    "A-League Men": "2",
    "Austrian Football Bundesliga": "2",
    "Belgian Pro League": "2",
    "División de Fútbol Profesional": "2",
    "First Professional Football League": "2",
    "Croatian Football League": "2",
    "Czech First League": "2",
    "Danish Superliga": "2",

    "Liga Profesional de Fútbol Argentina": "1",
    "Campeonato Brasileiro Série A": "1",
    "Canadian Premier League": "1",
    "Chilean Primera División": "1",
    "Chinese Football Association Super League": "1",
    "Categoría Primera A": "1",
    "Liga Profesional Ecuador": "1"
}

In [38]:
'''
Ponha o nome da competição. 
Abaixo estão alguns exemplos:

Premier League
La Liga
Ligue 1
Fußball-Bundesliga
Serie A
Liga Profesional de Fútbol Argentina
A-League Men
Austrian Football Bundesliga
Belgian Pro League
División de Fútbol Profesional
Campeonato Brasileiro Série A
First Professional Football League
Canadian Premier League
Chilean Primera División
Chinese Football Association Super League
Categoría Primera A
Croatian Football League
Czech First League
Danish Superliga
Liga Profesional Ecuador

'''

competition = input()

In [39]:
with SB() as sb:
    sb.open("https://fbref.com/en/comps/")
    
    sb.click('button:contains("Aceitar")')
    
    sb.scroll_to('a.sr_preset:contains("Club Competitions")')
    sb.sleep(2)
    
    sb.click('a.sr_preset:contains("Men")')
    sb.click('a.sr_preset:contains("Club Competitions")')
    sb.sleep(2)
    
    sb.scroll_to(f'th.left:contains("{competition}")')
    sb.sleep(2)
    
    href = sb.get_attribute(f'th.left a:contains("{competition}")', "href")
    sb.go_to(href)
    
    sb.scroll_to('th.left:contains("2023")')
    sb.sleep(2)
    
    if competitions_format.get(competition) == "2":
        anos.append(2025)
        anos = [ano-1 for ano in anos]
    
    for ano in anos:
        selector = f'a:contains("{ano}")'
        if sb.is_element_visible(selector):
            href = sb.get_attribute(selector, "href")
            if href:
                links[ano] = href
                print(f"🔗 Link para {ano}: {href}")
            else:
                print(f"⚠️ Elemento para {ano} não possui href!")
        else:
            print(f"❌ Elemento para {ano} não encontrado!")
    
    if sb.assert_element('//*[@id="seasons"]/tbody/tr[1]/th/a') and competitions_format.get(competition) == "1":
        links[2025] = sb.get_attribute('//*[@id="seasons"]/tbody/tr[1]/th/a', 'href')
        sb.sleep(2)
        print(f"🔗 Link para 2025: {links[2025]}")
        anos.append(2025)
        
    for ano, url in links.items():
        sb.execute_script(f"window.open('{url}', '_blank');")
        sb.sleep(2)
        sb.switch_to_window(-1)
        sb.sleep(2)
        for i in range(1, 21):
            xpath = f'//*[@id="stats_squads_standard_for"]/tbody/tr[{i}]/th/a'
            if sb.is_element_present(xpath):
                link = sb.get_attribute(xpath, 'href')
                team_name = link.split("/")[-1].replace("-Stats", "").replace("-", "_")
                teaminfo.setdefault(team_name, {})[ano] = link
    print(teaminfo)

🔗 Link para 2022: https://fbref.com/en/comps/20/2022-2023/2022-2023-Bundesliga-Stats
🔗 Link para 2023: https://fbref.com/en/comps/20/2023-2024/2023-2024-Bundesliga-Stats
🔗 Link para 2024: https://fbref.com/en/comps/20/2024-2025/2024-2025-Bundesliga-Stats
{'Augsburg': {2022: 'https://fbref.com/en/squads/0cdc4311/2022-2023/Augsburg-Stats', 2023: 'https://fbref.com/en/squads/0cdc4311/2023-2024/Augsburg-Stats', 2024: 'https://fbref.com/en/squads/0cdc4311/2024-2025/Augsburg-Stats'}, 'Bayern_Munich': {2022: 'https://fbref.com/en/squads/054efa67/2022-2023/Bayern-Munich-Stats', 2023: 'https://fbref.com/en/squads/054efa67/2023-2024/Bayern-Munich-Stats', 2024: 'https://fbref.com/en/squads/054efa67/2024-2025/Bayern-Munich-Stats'}, 'Bochum': {2022: 'https://fbref.com/en/squads/b42c6323/2022-2023/Bochum-Stats', 2023: 'https://fbref.com/en/squads/b42c6323/2023-2024/Bochum-Stats', 2024: 'https://fbref.com/en/squads/b42c6323/2024-2025/Bochum-Stats'}, 'Dortmund': {2022: 'https://fbref.com/en/squads/add

In [40]:
for ano, url in links.items():
  df_fbref_competition[ano] = pd.read_html(url, attrs={'id': 'stats_squads_standard_for'})[0]
  sb.sleep(5)

In [ ]:
with SB(headless=True) as sb:
    # Abrir qualquer link da primeira entrada do dicionário
    f_team = next(iter(teaminfo))
    f_ano = next(iter(teaminfo[f_team]))
    f_link = teaminfo[f_team][f_ano]

    sb.open(f_link)
    sb.sleep(1)  # esperar tabela carregar

    table_xpath = '/html/body/div[4]/div[6]/div[3]/div[3]/table'
    table_id = sb.get_attribute(table_xpath, "id")
    print(f"ID da tabela: {table_id}")

ID da tabela (pego apenas uma vez): stats_standard_20


In [57]:
for team_name, anos_links in teaminfo.items():
  df_fbref_teams.setdefault(team_name, {})
  for ano, link in anos_links.items():
    df_fbref_teams[team_name][ano] = pd.read_html(link, attrs={'id': f'{table_id}'})[0]
    sb.sleep(5)

In [58]:
for ano in anos:
  for team_name, anos_links in teaminfo.items():
    print(f'{team_name}: {ano}')

Augsburg: 2022
Bayern_Munich: 2022
Bochum: 2022
Dortmund: 2022
Eintracht_Frankfurt: 2022
Freiburg: 2022
Monchengladbach: 2022
Hertha_BSC: 2022
Hoffenheim: 2022
Koln: 2022
Bayer_Leverkusen: 2022
Mainz_05: 2022
RB_Leipzig: 2022
Schalke_04: 2022
Stuttgart: 2022
Union_Berlin: 2022
Werder_Bremen: 2022
Wolfsburg: 2022
Darmstadt_98: 2022
Heidenheim: 2022
Holstein_Kiel: 2022
St_Pauli: 2022
Augsburg: 2023
Bayern_Munich: 2023
Bochum: 2023
Dortmund: 2023
Eintracht_Frankfurt: 2023
Freiburg: 2023
Monchengladbach: 2023
Hertha_BSC: 2023
Hoffenheim: 2023
Koln: 2023
Bayer_Leverkusen: 2023
Mainz_05: 2023
RB_Leipzig: 2023
Schalke_04: 2023
Stuttgart: 2023
Union_Berlin: 2023
Werder_Bremen: 2023
Wolfsburg: 2023
Darmstadt_98: 2023
Heidenheim: 2023
Holstein_Kiel: 2023
St_Pauli: 2023
Augsburg: 2024
Bayern_Munich: 2024
Bochum: 2024
Dortmund: 2024
Eintracht_Frankfurt: 2024
Freiburg: 2024
Monchengladbach: 2024
Hertha_BSC: 2024
Hoffenheim: 2024
Koln: 2024
Bayer_Leverkusen: 2024
Mainz_05: 2024
RB_Leipzig: 2024
Scha

In [59]:
import os

for ano in anos:
    # Pega o DataFrame daquele ano
    df = df_fbref_competition[ano]
    df.columns = df.columns.get_level_values(1)

    # Criar pasta do campeonato
    folder_path = os.path.join(".", competition)
    os.makedirs(folder_path, exist_ok=True)

    # Nome do arquivo
    if competitions_format.get(competition) == "1":
        file_name = f"{competition}_{ano}.csv"
        file_path = os.path.join(folder_path, file_name)
        
    if competitions_format.get(competition) == "2":
        file_name = f"{competition}_{ano}-{ano+1}.csv"
        file_path = os.path.join(folder_path, file_name)

    # Salvar CSV
    df.to_csv(file_path, index=False, encoding="utf-8-sig")
    print(f"💾 Salvo: {file_path}")

💾 Salvo: .\Fußball-Bundesliga\Fußball-Bundesliga_2022-2023.csv
💾 Salvo: .\Fußball-Bundesliga\Fußball-Bundesliga_2023-2024.csv
💾 Salvo: .\Fußball-Bundesliga\Fußball-Bundesliga_2024-2025.csv


In [60]:
import os

for ano in anos:
    for team in df_fbref_teams:
        if ano in df_fbref_teams[team]:
            df_fbref_teams[team][ano].columns = df_fbref_teams[team][ano].columns.get_level_values(1)
            
            # Pasta do campeonato + time
            if competitions_format.get(competition) == "1":
                folder_path = os.path.join(".", competition, f'{ano}')
                os.makedirs(folder_path, exist_ok=True)
                
            elif competitions_format.get(competition) == "2":
                folder_path = os.path.join(".", competition, f'{ano}-{ano+1}')
                os.makedirs(folder_path, exist_ok=True)

            # Nome do arquivo
            if competitions_format.get(competition) == "1":
                file_name = f"{team}_{ano}.csv"
                file_path = os.path.join(folder_path, file_name)
                
            elif competitions_format.get(competition) == "2":
                file_name = f"{team}_{ano}-{ano+1}.csv"
                file_path = os.path.join(folder_path, file_name)

            # Salvar CSV
            df.to_csv(file_path, index=False, encoding="utf-8-sig")
            print(f"💾 Salvo: {file_path}")

💾 Salvo: .\Fußball-Bundesliga\2022-2023\Augsburg_2022-2023.csv
💾 Salvo: .\Fußball-Bundesliga\2022-2023\Bayern_Munich_2022-2023.csv
💾 Salvo: .\Fußball-Bundesliga\2022-2023\Bochum_2022-2023.csv
💾 Salvo: .\Fußball-Bundesliga\2022-2023\Dortmund_2022-2023.csv
💾 Salvo: .\Fußball-Bundesliga\2022-2023\Eintracht_Frankfurt_2022-2023.csv
💾 Salvo: .\Fußball-Bundesliga\2022-2023\Freiburg_2022-2023.csv
💾 Salvo: .\Fußball-Bundesliga\2022-2023\Monchengladbach_2022-2023.csv
💾 Salvo: .\Fußball-Bundesliga\2022-2023\Hertha_BSC_2022-2023.csv
💾 Salvo: .\Fußball-Bundesliga\2022-2023\Hoffenheim_2022-2023.csv
💾 Salvo: .\Fußball-Bundesliga\2022-2023\Koln_2022-2023.csv
💾 Salvo: .\Fußball-Bundesliga\2022-2023\Bayer_Leverkusen_2022-2023.csv
💾 Salvo: .\Fußball-Bundesliga\2022-2023\Mainz_05_2022-2023.csv
💾 Salvo: .\Fußball-Bundesliga\2022-2023\RB_Leipzig_2022-2023.csv
💾 Salvo: .\Fußball-Bundesliga\2022-2023\Schalke_04_2022-2023.csv
💾 Salvo: .\Fußball-Bundesliga\2022-2023\Stuttgart_2022-2023.csv
💾 Salvo: .\Fußball-Bu